<a href="https://colab.research.google.com/github/pranav-kaushal/Time-Series-Forecasting/blob/main/Time_Series_Forecasting_lesson_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Forecasting:
Time series forecasting occurs when you make scientific predictions based on historical time stamped data. It involves building models through historical analysis and using them to make observations and drive future strategic decision-making.

Predictions are made for new data when the actual outcome may not be known until some future date. The future is being predicted, but all prior observations are almost always treated equally.time series forecasting uses the information in a time series (perhaps with additional information) to forecast future values of that series.

Making predictions about the future is called extrapolation in the classical statistical handling of time series data.More modern fields focus on the topic and refer to it as time series forecasting.

Forecasting involves taking models fit on historical data and using them to predict future observations.


# Components of Time Series

1. Level. The baseline value for the series if it were a straight line.
2. Trend. The optional and often linear increasing or decreasing behavior of    the series over time.
3. Seasonality. The optional repeating patterns or cycles of behavior over time.
4. Noise. The optional variability in the observations that cannot be explained by the model.

Time series data often requires cleaning, scaling, and even transformation.

For example:

  Frequency. Perhaps data is provided at a frequency that is too high to model or is unevenly spaced through time requiring resampling for use in some models.
  Outliers. Perhaps there are corrupt or extreme outlier values that need to be identified and handled.
  Missing. Perhaps there are gaps or missing data that need to be interpolated or imputed.
